# Algorithmic Game Theory

Students:

- **Emanuele Conforti (252122)**
- **Jacopo Garofalo (252093)**
- **Gianmarco La Marca (252256)**

## **Assignment**

Consider a setting with a set N={1,2,…,n} of agents and with a set S = {s_1,...,s_m} of skills. Each agent i ∈ N is associated with a set of skills denoted by S_i ⊆ S. Note that there is no a-priori specified number of skills that each agent owns; for instance, an agent might have just one skill, while another might have all the available skills. In fact, there is a task t to be completed and all the skills in S are required to this end. Hence, agents are required to collaborate with each other and a number of strategic issues come into play.

For each of the following questions, implement in Python a method that can provide results for any possible pair N,S. Report then the results obtained over the instance, with 4 agents and 3 skills, graphically depicted below according to an intuitive notation (an edge means that the agent owns the corresponding skill).

In [58]:
agents = frozenset(['1', '2', '3', '4'])

skills_map = {
    '1': ['s1'],
    '2': ['s1', 's2'],
    '3': ['s1', 's2'],
    '4': ['s3'],
}

v = {
    frozenset(['1']): 0,
    frozenset(['2']): 0,
    frozenset(['3']): 0,
    frozenset(['4']): 0,
    frozenset(['1', '2']): 0,
    frozenset(['1', '3']): 0,
    frozenset(['1', '4']): 0,
    frozenset(['2', '3']): 0,
    frozenset(['2', '4']): 100,
    frozenset(['3', '4']): 100,
    frozenset(['1', '2', '3']): 0,
    frozenset(['1', '2', '4']): 100,
    frozenset(['1', '3', '4']): 100,
    frozenset(['2', '3', '4']): 100,
    frozenset(['1', '2', '3', '4']): 100,
}

## **Task 1**

Assume that completing the task t leads to a reward of 100$. Then, compute the Shapley value associated with the agents as a fair way to distribute that reward among them. And, finally, check whether the Shapley value is in the core of the coalitional game induced by the setting.

### **Shapley value**

In [59]:
from math import factorial
from itertools import combinations


# A function to generate the powerset: it returns a list of 2^(n-1) frozen sets
def powerset(List):
    subs = [frozenset(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return subs

# We now implement the Shapley value for a given player, using the second equation
def shapley_value(players, player, characteristic_function):
    power_set = powerset(players - {player})
    shapley = 0

    for c in power_set:
        first_part = factorial(len(c)) * factorial(len(players) - len(c) - 1)
        second_part = first_part / factorial(len(players))
        third_part = characteristic_function[c.union(player)] - characteristic_function[c]
        shapley += second_part * third_part

    return shapley

# This function returns a dictionary with the Shapley value for each player
def shapley(players, characteristic_function):
    # To get the grand coalition from the characteristic function, we can use the function max
    return {player: shapley_value(players, player, characteristic_function) for player in max(characteristic_function)}

In [60]:
shapley_values = shapley(agents, v)

print(shapley_values)

{'4': 66.66666666666666, '3': 16.666666666666664, '2': 16.666666666666664, '1': 0.0}


### **Core**

In [64]:
def is_positive(outcome):
    for i in outcome.values():
        if i < 0:
            return False
    return True

def is_efficient(outcome, characteristic_function, players):
    grand_coalition = characteristic_function[frozenset(players)]
    sum = 0
    for i in outcome.values():
        sum += i
    print("sum: ", sum)
    print("sum truncate: ", int(sum))
    print("correct sum rounding: ", round(sum))
    # we use the function round() for a correct rounding
    return round(sum) == grand_coalition


# This function checks if an outcome for a given game is stable
def is_stable(outcome, characteristic_function):
    for c in characteristic_function.keys():
        sum = 0
        for i in c:
            sum += outcome[i]
        if sum < characteristic_function[c]:
            print("c: ", c)
            print("sum: ", sum)
            print("characteristic_function[c]: ", characteristic_function[c])
            return False

    return True


# This function applies the definition of core to check whether an outcome is in the core of a game
def is_in_the_core(outcome, characteristic_function, players):
    print("is_positive: ", is_positive(outcome))
    print("--------------------------")
    print("is_efficient: ", is_efficient(outcome, characteristic_function, players))
    print("--------------------------")
    print("is_stable: ", is_stable(outcome, characteristic_function))
    print("--------------------------")
    return is_positive(outcome) and is_efficient(outcome, characteristic_function, players) and is_stable(outcome, characteristic_function)

In [65]:
# According to the example, the shapley values should be {'4': 66.66666666666666, '3': 16.666666666666664, '2': 16.666666666666664, '1': 0.0}
# Hence, it should be:
# POSITIVE: since every player outcome is >= 0,
# EFFICIENT: since the sum of all the outcomes is equal to the characteristic_function value of the grand coalition
# NOT STABLE: since there are some coalitions C such that, the sum of the shapley values of the players of C is < the value of the characteristic_function of C. Example: coalition {'4', '2'}
print(is_in_the_core(shapley_values, v, agents))

is_positive:  True
--------------------------
sum:  99.99999999999997
sum truncate:  99
correct sum rounding:  100
is_efficient:  True
--------------------------
c:  frozenset({'4', '2'})
sum:  83.33333333333331
characteristic_function[c]:  100
is_stable:  False
--------------------------
sum:  99.99999999999997
sum truncate:  99
correct sum rounding:  100
c:  frozenset({'4', '2'})
sum:  83.33333333333331
characteristic_function[c]:  100
False


## **Task 2**

Assume that each agent i∈ N might freely decide whether to join the group in order to complete the task. In particular, each agent incurs a fixed cost of c_i to join the group and s/he is selfish interested, so that s/he would like that the revenue s/he gets by collaboration covers this expense (in the example, let c_1 = c_2 = 10$, c_3 = 20$, c_4 = 40$. Assume that the revenue is divided according to the Shapley value of the coalitional game induced by the agents that join the group (as in item 1). Then, check whether the resulting strategic setting, where each agent has two actions (joint, not join), admits a pure Nash equilibrium and computes one, if any.


## **Task 3**

Assume that all agents participate to the setting, but they might cheat on the cost c_i, and consider a setting where a mechanism has to identify a group of agents that is capable of completing the task with the minimum overall cost. Then, compute a payment scheme that provides incentives to truthfully report such costs.